In [ ]:
# Initiate GPU runtime

import tensorflow as tf
tf.test.gpu_device_name()

import torch
torch.cuda.get_device_name(0)

!rm -rf sample_data

In [ ]:
# Install Guppy (Oxford Nanopore Technologies)

%%shell

GUPPY_VERSION=6.5.7 # Archived Software

wget https://cdn.oxfordnanoportal.com/software/analysis/ont-guppy_"$GUPPY_VERSION"_linux64.tar.gz -O ont-guppy.tar.gz
tar -vzxf ont-guppy.tar.gz; rm -rf ont-guppy.tar.gz
./ont-guppy/bin/guppy_basecaller --version

In [ ]:
# Import Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!ls gdrive/MyDrive

In [ ]:
# Basecall and Demultiplex process

%%shell

# Specify the name of the folder (i.e., 20240411_1608_MN22990_FAT84733_f175dd69/fast5)
DIR="20240411_1608_MN22990_FAT84733_f175dd69/fast5"

# Specify the barcode kit used for library preparation
BARKIT="EXP-NBD114"

PREFIX="$(echo "$IDIR" | awk -F/ '{print $1}')"

# Specify the accurate basecalling model - fast (fast), high (hac) or super (sup) accuracy
# dna_r9.4.1_450bps_fast.cfg
# dna_r9.4.1_450bps_hac.cfg
# dna_r9.4.1_450bps_sup.cfg
./ont-guppy/bin/guppy_basecaller -r --device cuda:0 --verbose_logs \
-i "$IDIR" -s "$PREFIX"_GUPPY-BASECALL --min_qscore 8 \
-c "ont-guppy/data/dna_r9.4.1_450bps_fast.cfg" \
--gpu_runners_per_device 16 --cpu_threads_per_caller $(nproc)

./ont-guppy/bin/guppy_barcoder -r --trim_adapters --barcode_kits "$BARKIT" \
-i "$PREFIX"_GUPPY-FAST -s "$PREFIX"_GUPPY-DEMUX -t $(nproc)